In [2]:
!pip install datasets

In [ ]:
https://github.com/sindhubhaies/Fake-News-Detection
https://github.com/sindhubhaies/forest_Fire_Prediction
https://github.com/sindhubhaies/Tweet-Sentiment-Analysis
https://github.com/sindhubhaies/Credit_Card_Fraud_Detection

In [ ]:
Tweet Sentiment Analysis : https://www.linkedin.com/posts/sindhubhai-es_codxo-codxo-datascience-activity-7230697163603816448-H3N8?utm_source=share&utm_medium=member_desktop


Credit Card Fraud Detection : https://www.linkedin.com/posts/sindhubhai-es_codxo-codxo-datascience-activity-7230700149449052162-zX_R?utm_source=share&utm_medium=member_desktop


Forest Fire Detection : https://www.linkedin.com/posts/sindhubhai-es_codxo-codxo-forestfireprediction-activity-7230695877714042880-Auhz?utm_source=share&utm_medium=member_desktop


Fake News Detection :  https://www.linkedin.com/posts/sindhubhai-es_codxo-grateful-learning-activity-7230694739681943554-VM4x?utm_source=share&utm_medium=member_desktop

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve


In [4]:
# Load the dataset
ds = load_dataset("dazzle-nu/CIS435-CreditCardFraudDetection")
df = pd.DataFrame(ds['train'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1048575 [00:00<?, ? examples/s]

In [5]:
# Display basic information about the dataset
print(df.head())
print(df.info())

   Unnamed: 0 trans_date_trans_time        cc_num  \
0           0           1/1/19 0:00  2.703190e+15   
1           1           1/1/19 0:00  6.304230e+11   
2           2           1/1/19 0:00  3.885950e+13   
3           3           1/1/19 0:01  3.534090e+15   
4           4           1/1/19 0:03  3.755340e+14   

                             merchant       category     amt      first  \
0          fraud_Rippin, Kub and Mann       misc_net    4.97   Jennifer   
1     fraud_Heller, Gutmann and Zieme    grocery_pos  107.23  Stephanie   
2                fraud_Lind-Buckridge  entertainment  220.11     Edward   
3  fraud_Kutch, Hermiston and Farrell  gas_transport   45.00     Jeremy   
4                 fraud_Keeling-Crist       misc_pos   41.96      Tyler   

      last gender                        street  ... city_pop  \
0    Banks      F                561 Perry Cove  ...     3495   
1     Gill      F  43039 Riley Greens Suite 393  ...      149   
2  Sanchez      M      594 White Da

In [6]:
# Drop unwanted columns
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 23', '6006', 'first', 'last', 'street', 'city', 'state', 'zip'])

In [7]:
# Convert 'trans_date_trans_time' and 'dob' to datetime format
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'])

<ipython-input-7-49a283d12a21>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
<ipython-input-7-49a283d12a21>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['dob'] = pd.to_datetime(df['dob'])


In [8]:
# Feature Engineering
df['transaction_hour'] = df['trans_date_trans_time'].dt.hour
df['age'] = (df['trans_date_trans_time'] - df['dob']).dt.days // 365

In [9]:
# Drop the original datetime columns
df = df.drop(columns=['trans_date_trans_time', 'dob'])

In [ ]:
# Encoding categorical variables
categorical_cols = ['merchant', 'category', 'gender', 'job', 'trans_num']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

In [ ]:
# Define features (X) and target (y)
X = df.drop(columns=['is_fraud'])
y = df['is_fraud']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model training - Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Predictions
y_pred = model.predict(X_test_scaled)

# Model evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# ROC Curve and AUC
y_prob = model.predict_proba(X_test_scaled)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = roc_auc_score(y_test, y_prob)

plt.figure(figsize=(10, 7))
plt.plot(fpr, tpr, label=f'ROC Curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

<ipython-input-16-a741c6fa20ca>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
<ipython-input-16-a741c6fa20ca>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['dob'] = pd.to_datetime(df['dob'])
